In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Reference
https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
# extract the whole website
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [4]:
# extract the whole table
My_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
# extract all things about postcodes, borough and neighborhood
lists = My_table.findAll('td')
newlists = []
for item in lists:
    tmpstr = str(item)
    if tmpstr.find('</a>') == -1:
        newlists.append(tmpstr[tmpstr.find('<td>')+4:tmpstr.find('</td>')].replace('\n',''))
    else:
        newlists.append(tmpstr[tmpstr.find('">')+2:tmpstr.find('</a>')].replace('\n',''))
        
newlists[:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [6]:
# extract columns
postcodes = newlists[0::3]
borough = newlists[1::3]
neighborhood = newlists[2::3]

In [7]:
neighborhood[-3:]

['Royal York South West', 'South of Bloor', 'Not assigned']

#### Solve:
#### Ignore cells with a borough that is Not assigned.
#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 


In [8]:
p,b,n = [],[],[]
for i in range(len(postcodes)):
    if borough[i] != 'Not assigned':
        p.append(postcodes[i])
        b.append(borough[i])
        if neighborhood[i] == 'Not assigned':
            n.append(borough[i])
        else:
            n.append(neighborhood[i])

In [9]:
from operator import itemgetter

#### Solve:
#### Combine Multiple rows with the same postcodes and boroughs to one row of neighborhood

In [14]:
POST,BOR,NEIGH = [],[],[]
for i in range(len(p)):
    if p[i] not in POST:
        POST.append(p[i])
        BOR.append(b[i])
        tmpid = [j for j,x in enumerate(p) if x==p[i]]
        tmgetter = itemgetter(*tmpid)
        tmplist = tmgetter(n)
        if isinstance(tmplist,str):
            NEIGH.append(tmplist)
        elif isinstance(tmplist,tuple):
            NEIGH.append(','.join(list(tmplist)))

In [15]:
df = pd.DataFrame()
df['PostalCode'] = POST
df['Borough'] = BOR
df['Neighborhood'] = NEIGH

#### Show the same content from the instruction

In [22]:
df.loc[df['PostalCode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M1B,Scarborough,"Rouge,Malvern"
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Maryvale,Wexford"


#### Dataframe shape

In [18]:
df.shape

(103, 3)

#### Geographical coordinates of each postal code

In [25]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [26]:
# So slow
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(POST))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

#### Read csv

In [30]:
spadata = pd.read_csv('http://cocl.us/Geospatial_data')

#### Rename the column Postal Code

In [33]:
spadata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#### Merge two dataframes

In [34]:
df2 = pd.merge(df,spadata)

#### Show the wanted PostalCode

In [35]:
want = ['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']
df2.loc[df2['PostalCode'].isin(want)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
